In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from collections import Counter
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import xgboost
from sklearn.metrics import r2_score

import re


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/3rd-ml100marathon-midterm/train_data.csv
/kaggle/input/3rd-ml100marathon-midterm/enron61702insiderpay.pdf
/kaggle/input/3rd-ml100marathon-midterm/sample_submission.csv
/kaggle/input/3rd-ml100marathon-midterm/test_features.csv


In [2]:
train = pd.read_csv('../input/3rd-ml100marathon-midterm/train_data.csv')
test = pd.read_csv('../input/3rd-ml100marathon-midterm/test_features.csv')
submission = pd.read_csv('../input/3rd-ml100marathon-midterm/sample_submission.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 22 columns):
name                         113 non-null object
bonus                        61 non-null float64
deferral_payments            28 non-null float64
deferred_income              34 non-null float64
director_fees                13 non-null float64
email_address                83 non-null object
exercised_stock_options      81 non-null float64
expenses                     73 non-null float64
from_messages                65 non-null float64
from_poi_to_this_person      65 non-null float64
from_this_person_to_poi      65 non-null float64
loan_advances                2 non-null float64
long_term_incentive          49 non-null float64
other                        69 non-null float64
poi                          113 non-null bool
restricted_stock             82 non-null float64
restricted_stock_deferred    10 non-null float64
salary                       73 non-null float64
shared_receipt_wi

In [4]:
drop_list = ['name']
train.drop(drop_list, axis=1, inplace=True)
test.drop(drop_list, axis=1, inplace=True)



In [5]:
train['email_address'].loc[train['email_address'].notnull()] = 1
train.head(20)



/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,NaN,-3504386.0,NaN,1,19794175.0,46950.0,18.0,42.0,4.0,...,1617011.0,174839.0,True,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,NaN,NaN,NaN,1,19250000.0,29336.0,108.0,88.0,30.0,...,1920000.0,22122.0,True,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,NaN,-4167.0,NaN,1,1624396.0,22884.0,39.0,13.0,14.0,...,NaN,1573324.0,True,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,NaN,NaN,NaN,1,NaN,118134.0,NaN,NaN,NaN,...,602671.0,907502.0,True,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,1250000.0,NaN,-262500.0,NaN,1,NaN,35818.0,144.0,199.0,25.0,...,375304.0,486.0,True,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0
5,NaN,10259.0,NaN,NaN,1,30766064.0,77978.0,NaN,NaN,NaN,...,NaN,2856.0,True,NaN,NaN,NaN,NaN,NaN,91093.0,30766064.0
6,NaN,NaN,NaN,NaN,1,8308552.0,53947.0,NaN,NaN,NaN,...,NaN,147950.0,True,3576206.0,NaN,158403.0,NaN,NaN,360300.0,11884758.0
7,1200000.0,27610.0,-144062.0,NaN,1,NaN,16514.0,40.0,240.0,11.0,...,NaN,101740.0,True,698242.0,NaN,288542.0,1132.0,1758.0,1490344.0,698242.0
8,7000000.0,202911.0,-300000.0,NaN,1,34348384.0,99832.0,36.0,123.0,16.0,...,3600000.0,10359729.0,True,14761694.0,NaN,1072321.0,2411.0,4273.0,103559793.0,49110078.0
9,600000.0,NaN,NaN,NaN,1,384728.0,125978.0,16.0,52.0,6.0,...,71023.0,200308.0,True,393818.0,NaN,274975.0,874.0,873.0,1272284.0,778546.0


In [6]:
test['email_address'].loc[test['email_address'].notnull()] = 1
test.head(20)

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,5249999.0,2144013.0,-2334434.0,NaN,1,953136.0,17355.0,484.0,228.0,108.0,NaN,NaN,210698.0,157569.0,NaN,213999.0,5521.0,7991.0,5501630.0,1110705.0
1,1350000.0,NaN,-833.0,NaN,1,NaN,65907.0,27.0,140.0,15.0,NaN,974293.0,1621.0,252055.0,NaN,278601.0,1593.0,1858.0,2669589.0,252055.0
2,1500000.0,NaN,-3117011.0,NaN,1,5538001.0,34039.0,32.0,32.0,21.0,NaN,1617011.0,11350.0,853064.0,NaN,243293.0,1035.0,1045.0,288682.0,6391065.0
3,3000000.0,NaN,NaN,NaN,1,2291113.0,86174.0,3069.0,66.0,609.0,NaN,1294981.0,1661.0,1323148.0,NaN,365163.0,2097.0,3093.0,4747979.0,3614261.0
4,1000000.0,NaN,-235000.0,NaN,1,NaN,30674.0,49.0,58.0,12.0,NaN,350000.0,307895.0,2502063.0,NaN,415189.0,1585.0,1892.0,1868758.0,2502063.0
5,1700000.0,NaN,NaN,NaN,1,NaN,98849.0,27.0,40.0,1.0,NaN,69223.0,1936.0,441096.0,NaN,211788.0,900.0,1320.0,2081796.0,441096.0
6,2000000.0,6426990.0,-3367011.0,NaN,1,10433518.0,86987.0,21.0,242.0,6.0,2000000.0,1617011.0,7427621.0,4188667.0,NaN,1060932.0,2979.0,3275.0,17252530.0,14622185.0
7,NaN,NaN,-98784.0,98784.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32460.0,-32460.0,NaN,NaN,NaN,NaN,NaN
8,800000.0,NaN,-300000.0,NaN,1,765920.0,96268.0,22.0,188.0,11.0,NaN,NaN,891.0,315068.0,NaN,278601.0,772.0,865.0,875760.0,1080988.0
9,100000.0,51365.0,-200000.0,NaN,NaN,4160672.0,8409.0,NaN,NaN,NaN,NaN,156250.0,202052.0,201483.0,-140264.0,76399.0,NaN,NaN,394475.0,4221891.0


In [7]:

train = train.fillna(0)
test = test.fillna(0)


In [8]:
train['email_address'].astype('int64')
test['email_address'].astype('int64')

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     0
8     1
9     0
10    1
11    1
12    1
13    0
14    1
15    1
16    1
17    1
18    1
19    1
20    0
21    1
22    1
23    1
24    1
25    1
26    1
27    0
28    1
29    1
30    1
31    1
32    1
Name: email_address, dtype: int64

In [9]:
numeric_cols = [col for col in train.columns if train[col].dtype in [np.float64, np.int64]]
numeric_cols

['bonus',
 'deferral_payments',
 'deferred_income',
 'director_fees',
 'email_address',
 'exercised_stock_options',
 'expenses',
 'from_messages',
 'from_poi_to_this_person',
 'from_this_person_to_poi',
 'loan_advances',
 'long_term_incentive',
 'other',
 'restricted_stock',
 'restricted_stock_deferred',
 'salary',
 'shared_receipt_with_poi',
 'to_messages',
 'total_payments',
 'total_stock_value']

In [10]:
train.head(20)

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,0.0,-3504386.0,0.0,1,19794175.0,46950.0,18.0,42.0,4.0,...,1617011.0,174839.0,True,2748364.0,0.0,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,0.0,0.0,0.0,1,19250000.0,29336.0,108.0,88.0,30.0,...,1920000.0,22122.0,True,6843672.0,0.0,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,0.0,-4167.0,0.0,1,1624396.0,22884.0,39.0,13.0,14.0,...,0.0,1573324.0,True,869220.0,0.0,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,0.0,0.0,0.0,1,0.0,118134.0,0.0,0.0,0.0,...,602671.0,907502.0,True,985032.0,0.0,224305.0,0.0,0.0,2652612.0,985032.0
4,1250000.0,0.0,-262500.0,0.0,1,0.0,35818.0,144.0,199.0,25.0,...,375304.0,486.0,True,126027.0,0.0,240189.0,2188.0,2598.0,1639297.0,126027.0
5,0.0,10259.0,0.0,0.0,1,30766064.0,77978.0,0.0,0.0,0.0,...,0.0,2856.0,True,0.0,0.0,0.0,0.0,0.0,91093.0,30766064.0
6,0.0,0.0,0.0,0.0,1,8308552.0,53947.0,0.0,0.0,0.0,...,0.0,147950.0,True,3576206.0,0.0,158403.0,0.0,0.0,360300.0,11884758.0
7,1200000.0,27610.0,-144062.0,0.0,1,0.0,16514.0,40.0,240.0,11.0,...,0.0,101740.0,True,698242.0,0.0,288542.0,1132.0,1758.0,1490344.0,698242.0
8,7000000.0,202911.0,-300000.0,0.0,1,34348384.0,99832.0,36.0,123.0,16.0,...,3600000.0,10359729.0,True,14761694.0,0.0,1072321.0,2411.0,4273.0,103559793.0,49110078.0
9,600000.0,0.0,0.0,0.0,1,384728.0,125978.0,16.0,52.0,6.0,...,71023.0,200308.0,True,393818.0,0.0,274975.0,874.0,873.0,1272284.0,778546.0


In [11]:
X_columns = [col for col in train.columns]
X_columns.remove('poi')

In [12]:
X_columns

['bonus',
 'deferral_payments',
 'deferred_income',
 'director_fees',
 'email_address',
 'exercised_stock_options',
 'expenses',
 'from_messages',
 'from_poi_to_this_person',
 'from_this_person_to_poi',
 'loan_advances',
 'long_term_incentive',
 'other',
 'restricted_stock',
 'restricted_stock_deferred',
 'salary',
 'shared_receipt_with_poi',
 'to_messages',
 'total_payments',
 'total_stock_value']

In [13]:
sc=MinMaxScaler()
train[X_columns]=sc.fit_transform(train[X_columns])
test[X_columns]=sc.transform(test[X_columns])

In [14]:
train

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,0.21875,0.03342,0.000000,0.0,1.0,0.576277,0.205234,0.001253,0.079545,0.009732,...,0.314261,0.016877,True,0.186182,0.927036,0.378522,0.190855,0.059740,0.004877,0.459506
1,0.70000,0.03342,1.000000,0.0,1.0,0.560434,0.128238,0.007517,0.166667,0.072993,...,0.373146,0.002135,True,0.463610,0.927036,1.000000,0.451071,0.239422,0.083843,0.531751
2,0.02500,0.03342,0.998811,0.0,1.0,0.047292,0.100034,0.002714,0.024621,0.034063,...,0.000000,0.151869,True,0.058883,0.927036,0.190634,0.020102,0.014852,0.019350,0.051628
3,0.10000,0.03342,1.000000,0.0,1.0,0.000000,0.516403,0.000000,0.000000,0.000000,...,0.117127,0.087599,True,0.066729,0.927036,0.201848,0.000000,0.000000,0.025614,0.020937
4,0.15625,0.03342,0.925094,0.0,1.0,0.000000,0.156573,0.010022,0.376894,0.060827,...,0.072939,0.000047,True,0.008537,0.927036,0.216142,0.483322,0.171496,0.015829,0.003461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,0.06250,0.03342,1.000000,0.0,1.0,0.000000,0.072188,0.000000,0.000000,0.000000,...,0.000000,0.034371,False,0.034666,0.927036,0.215523,0.000000,0.000000,0.010739,0.011308
109,0.00000,0.03342,1.000000,0.0,1.0,0.000000,0.215454,0.002854,0.024621,0.002433,...,0.000000,0.000000,False,0.000000,0.927036,0.000000,0.168102,0.098092,0.000476,0.000897
110,0.03750,0.03342,1.000000,0.0,1.0,0.000838,0.138366,0.004663,0.013258,0.000000,...,0.000000,0.000000,False,0.000000,0.927036,0.589455,0.160813,0.059278,0.013390,0.001483
111,0.00000,0.03342,1.000000,0.0,1.0,0.000000,0.002076,0.001322,0.007576,0.017032,...,0.000000,0.000000,False,0.150206,0.927036,0.000000,0.104042,0.037824,0.000005,0.046006


In [15]:
from  sklearn.model_selection import train_test_split
train, valid = train_test_split(train, train_size=0.8, random_state=1212)
X_train = train[X_columns]
X_valid = valid[X_columns]
Y_train = train['poi']
Y_valid = valid['poi']
X_test = test[X_columns]




In [16]:
# cpuconf = { 
#     'max_depth': 10,
#     'n_estimators': 2000,#2000,
#     'learning_rate': 0.01,#0.01,
#     'gamma': 0,
#     'subsample': 0.7,
#     'n_jobs': 4,
#     'objective': 'reg:squarederror',
#     'alpha': 0.00006,
#     'eval_metric':'auc'
# }

# xgb = xgboost.XGBRegressor(**cpuconf)

# xgb.fit(X_train, Y_train)
# predictions = xgb.predict(X_valid)

# print(f'R2 Score: {r2_score(Y_valid, predictions)}')


In [17]:
# X = test[X_columns]

# predictions = xgb.predict(X)
# my_submission = pd.DataFrame({'name':submission.name,'poi': predictions})
# my_submission.to_csv('submission1.csv', index=False)
# print(predictions)

In [18]:
# Train xgboost
dtrain = xgboost.DMatrix(X_train, Y_train)
dvalid = xgboost.DMatrix(X_valid, Y_valid)

param = {'max_depth':10, 'eta':0.01, 'objective':'binary:logistic', 'eval_metric':'auc', 'seed':42}
num_round = 500
bst = xgboost.train(param, dtrain, num_round, [(dvalid, 'valid'), (dtrain, 'train')] )

[0]	valid-auc:0.575	train-auc:0.925
[1]	valid-auc:0.575	train-auc:0.925
[2]	valid-auc:0.575	train-auc:0.925
[3]	valid-auc:0.575	train-auc:0.925
[4]	valid-auc:0.575	train-auc:0.925
[5]	valid-auc:0.575	train-auc:0.925
[6]	valid-auc:0.575	train-auc:0.925
[7]	valid-auc:0.575	train-auc:0.925
[8]	valid-auc:0.575	train-auc:0.925
[9]	valid-auc:0.575	train-auc:0.925
[10]	valid-auc:0.575	train-auc:0.925
[11]	valid-auc:0.575	train-auc:0.925
[12]	valid-auc:0.575	train-auc:0.925
[13]	valid-auc:0.575	train-auc:0.925
[14]	valid-auc:0.575	train-auc:0.925
[15]	valid-auc:0.575	train-auc:0.925
[16]	valid-auc:0.575	train-auc:0.925
[17]	valid-auc:0.533333	train-auc:0.97375
[18]	valid-auc:0.533333	train-auc:0.97375
[19]	valid-auc:0.533333	train-auc:0.975
[20]	valid-auc:0.533333	train-auc:0.975
[21]	valid-auc:0.516667	train-auc:0.975
[22]	valid-auc:0.516667	train-auc:0.975
[23]	valid-auc:0.533333	train-auc:0.975
[24]	valid-auc:0.516667	train-auc:0.975
[25]	valid-auc:0.516667	train-auc:0.975
[26]	valid-auc:0.

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/opt/conda/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[134]	valid-auc:0.55	train-auc:0.989375
[135]	valid-auc:0.55	train-auc:0.989375
[136]	valid-auc:0.55	train-auc:0.9925
[137]	valid-auc:0.55	train-auc:0.99375
[138]	valid-auc:0.566667	train-auc:0.99375
[139]	valid-auc:0.55	train-auc:0.99375
[140]	valid-auc:0.566667	train-auc:0.99375
[141]	valid-auc:0.566667	train-auc:0.99375
[142]	valid-auc:0.566667	train-auc:0.99375
[143]	valid-auc:0.566667	train-auc:0.99375
[144]	valid-auc:0.566667	train-auc:0.99375
[145]	valid-auc:0.566667	train-auc:0.99375
[146]	valid-auc:0.566667	train-auc:0.99375
[147]	valid-auc:0.566667	train-auc:0.99375
[148]	valid-auc:0.566667	train-auc:0.99375
[149]	valid-auc:0.55	train-auc:0.99375
[150]	valid-auc:0.55	train-auc:0.99375
[151]	valid-auc:0.55	train-auc:0.99375
[152]	valid-auc:0.55	train-auc:0.99375
[153]	valid-auc:0.55	train-auc:0.99375
[154]	valid-auc:0.55	train-auc:0.99375
[155]	valid-auc:0.566667	train-auc:0.99375
[156]	valid-auc:0.566667	train-auc:0.99375
[157]	valid-auc:0.566667	train-auc:0.99375
[158]	valid

In [19]:

dtest = xgboost.DMatrix(X_test)
predictions = bst.predict(dtest)
my_submission = pd.DataFrame({'name':submission.name,'poi': predictions})
my_submission.to_csv('submission5.csv', index=False)
print(predictions)

[0.1674928  0.22666614 0.53519756 0.04592776 0.4230995  0.1623047
 0.76700264 0.01168408 0.15242325 0.08060973 0.04573634 0.01008209
 0.01008209 0.01008209 0.8313503  0.01008209 0.03040005 0.03408828
 0.08935082 0.01370946 0.7452033  0.40236586 0.07234295 0.02563659
 0.01115013 0.0138537  0.0252787  0.01384166 0.13109054 0.23171255
 0.01583614 0.13275222 0.592919  ]
